# <span style="color:#336699">SER-347 - Trabalho final</span>
<hr style="border:2px solid #0077b9;">

# <span style="color:#336699">Detecção de alvos artificiais em ambientes marítimos por meio de imagens SAR</span>

- Leonan Entringer Falqueto
- Billy Edy Mendes

## Objetivo: 
    
   Pretende-se estruturar um programa, utilizando a linguagem python, para aplicar a técnica de _threshold_ e identificar objetos artificiais em ambiente marítimo.

## __Primeira fase__: abrir a imagem SAR.

__Desafio__: identificar a codificação do arquivo (configurações e metadados) para abrir a imagem com Python.

Foram utilizadas imagens _full polarimétricas_ do Satélite ALOS PALSAR já com um nível de processamento 
(Compressão de pulsos em _range_ e azimute, L-1.1) e a biblioteca GDAL para ler as bandas.

In [1]:
# importar bibliotecas
from osgeo import gdal
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import math 
import cmath as cm
import datetime as dt
import os
import shutil
import glob
import sys

# importar constantes
from gdalconst import *

# informar o uso de exceções
gdal.UseExceptions()

# mostrar versão instalada
print ("Para esse programa, foram usadas as seguintes versões de bibliotecas, da Anaconda e do python:",
       "\n","Gdal 2.2.2, Matplotlib 2.2.2, Numpy 1.14.2, Conda 4.5.4 e Python 3.6.5", "\n\n",
      "Confira suas versões instaladas e seu ambiente de trabalho:")
print ("Gdal", gdal.__version__)
print ("Matplotlib", matplotlib.__version__)
print ("Numpy", np.__version__)
!conda --version
!python --version
!conda info --env

Para esse programa, foram usadas as seguintes versões de bibliotecas, da Anaconda e do python: 
 Gdal 2.2.2, Matplotlib 2.2.2, Numpy 1.14.2, Conda 4.5.4 e Python 3.6.5 

 Confira suas versões instaladas e seu ambiente de trabalho:
Gdal 2.2.2
Matplotlib 2.2.2
Numpy 1.14.2
conda 4.5.4


Python 3.6.5 :: Anaconda, Inc.


# conda environments:
#
base                     C:\Users\Falqueto\Anaconda3
geospatial            *  C:\Users\Falqueto\Anaconda3\envs\geospatial
ser347                   C:\Users\Falqueto\Anaconda3\envs\ser347



Ler a imagem:

In [2]:
# Criar o dataset abrindo o arquivo para leitura
# o 'r' antes do caminho serve para entender a string raw para não ter problemas com a \ (caracter especial).

filename = r"C:\SAR_data\ALPSRP173876680-L1.1-Merluza\ALPSRP173876680-L1.1\VOL-ALPSRP173876680-P1.1__A"

# Utilizando o comando try-except da biblioteca sys, será executada a leitura do arquivo no modo Read Only para não afetar
# a imagem original:

try:
    dataset = gdal.Open(filename, GA_ReadOnly)
    print ("Arquivo aberto com sucesso")
except:
    print("Erro na abertura do arquivo")

dataset = gdal.Open(filename, GA_ReadOnly)
if not dataset:
    ...

Arquivo aberto com sucesso


Obter os metadados:

In [3]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                             dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
geotransform = dataset.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Resolution = (Range: {}, Azimuth: {})".format(geotransform[1], geotransform[5]))

Driver: JAXAPALSAR/JAXA PALSAR Product Reader (Level 1.1/1.5)
Size is 1088 x 18432 x 4
Projection is 
Origin = (0.0, 0.0)
Resolution = (Range: 1.0, Azimuth: 1.0)


Ler as bandas:

In [4]:
# Função para extrair alguns dados estatísticos da Banda:
def Statistic_raster(banda):
    st = banda.GetStatistics(False,True)
    print("Min=%.2f, Max=%.2f, Mean=%.2f, Std=%.2f" % (st[0], st[1], st[2], st[3] ))
    #min_banda = banda.GetMinimum()
    #max_banda = banda.GetMaximum()
    #if not min_banda or not max_banda:
    #    (min_banda,max_banda) = banda.ComputeRasterMinMax(True)
    #print("Min={:.3f}, Max={:.3f}".format(min_banda,max_banda))
    return None

elif bandas == 2:
    print("É um arquivo dual pol")
    for i in range(bandas):
        banda = dataset.GetRasterBand(i+1)
        print("Band {} Type={}".format((i+1, gdal.GetDataTypeName(banda.DataType)))
              
elif bandas == 1:
    print("É um arquivo single pol")
    banda = dataset.GetRasterBand(1)
    print("Band SLC Type={}".format(gdal.GetDataTypeName(banda.DataType)))

In [19]:
polarizacoes = ["array_HH", "array_HV", "array_VH", "array_VV"]
dual_pol = ["copol", "crosspol"]
bandas = dataset.RasterCount
if bandas == 4:
    print("É um arquivo full pol")
    for i in range(bandas):
        banda = dataset.GetRasterBand(i+1)
        print("Band {} Type={}".format(polarizacoes[i] , gdal.GetDataTypeName(banda.DataType)))
        
elif bandas == 2:
    print("É um arquivo dual pol")
    for i in range(bandas):
        print("Band {} Type={}".format((dual_pol[i], gdal.GetDataTypeName(banda.DataType)))
              
elif bandas == 1:
    print("É um arquivo single pol")
    banda = dataset.GetRasterBand(1)
    print("Band SLC Type={}".format(gdal.GetDataTypeName(banda.DataType)))
        
else:
    print("A imagem possui outro tipo de polarização ou não é possível fazer sua leitura")

dataset = None

SyntaxError: invalid syntax (<ipython-input-19-604cb79846ed>, line 15)

In [18]:

#Transformar as bandas em um objeto numpy (Uma array)
array_HH = band11.ReadAsArray()
array_VV = band22.ReadAsArray()
dataset = None
#Ler a banda HH:
print("Banda HH:",'\n', array_HH,'\n')

#Ler a primeira linha da banda HH:
print("Primeira linha da banda HH:",'\n', array_HH[0],'\n')

#Ler o primeiro pixel da banda HH:
array_HH[0][0]

#Obter a amplitude e fase do primeiro pixel:
print("O primeiro pixel da banda HH possui:", "\n",
         'Amplitude:{} e fase: {}'.format(cm.polar(array_HH[0][0])[0], cm.polar(array_HH[0][0])[1]),'\n')
       


NameError: name 'array_HH' is not defined

In [ ]:
#imagem amplitude:
img_amplitude = np.absolute(array_HH)
img_intensidade = img_amplitude**2



#mostrar a imagem:
plt.figure( figsize=(3, 10 ))
plt.imshow(img_amplitude, cmap='gray')
plt.show()
plt.imshow(img_intensidade, cmap='gray')
plt.show()


In [ ]:
# verificar algumas propriedades das bandas, como o histograma
plt.plot(band11.GetHistogram())
plt.show()

display:

__Segunda fase__: aplicar um filtro para suavizar o efeito _Speckle_ típico de imagens SAR.

__Desafio__: construir uma janela de convolução para aplicar a fórmula do filtro escolhido.

__Terceira fase__: Calcular o parâmetro escolhido (grau de polarização, intensidade, etc)

__Desafio__: utilizar um parâmetro que utiliza dados de mais de uma banda (corregistrar as bandas)

__Quarta fase__: Aplicar um limiar para separar os alvos.

__Desafio__: utilizar um threshold adaptativo e criar uma máscara.

In [ ]:
import struct
tuple_of_floats = struct.unpack('f' * band.XSize, scanline)
palsarCal = '10^(2*log10(b1) - 8.3)'

Verificando se é possível criar um arquivo em algum formato específico.

Nesse caso, o GeoTiff:

In [ ]:
fileformat = "GTiff"
driver = gdal.GetDriverByName(fileformat)
metadata = driver.GetMetadata()
if metadata.get(gdal.DCAP_CREATE) == "YES":
    print("Driver {} supports Create() method.".format(fileformat))
    
if metadata.get(gdal.DCAP_CREATE) == "YES":
    print("Driver {} supports CreateCopy() method.".format(fileformat))